In [ ]:
from google.colab import drive
from google.colab import files

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd ''/content/drive/Shared drives/Team 1 earthquake''

/content/drive/Shared drives/Team 1 earthquake


In [ ]:
pip install --pre --extra-index https://pypi.anaconda.org/scipy-wheels-nightly/simple scikit-learn

Looking in indexes: https://pypi.org/simple, https://pypi.anaconda.org/scipy-wheels-nightly/simple


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, minmax_scale, MaxAbsScaler, Normalizer, QuantileTransformer, PowerTransformer
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE, mutual_info_classif
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from time import time
from matplotlib.colors import ListedColormap
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from scipy.stats import randint
sns.set()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df_train=pd.read_csv("train_values.csv")
df_train = pd.get_dummies(data=df_train, columns=['land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type', 'other_floor_type', 'position', 'plan_configuration', 'legal_ownership_status'])
df_train.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,land_surface_condition_n,land_surface_condition_o,land_surface_condition_t,foundation_type_h,foundation_type_i,foundation_type_r,foundation_type_u,foundation_type_w,roof_type_n,roof_type_q,roof_type_x,ground_floor_type_f,ground_floor_type_m,ground_floor_type_v,ground_floor_type_x,ground_floor_type_z,other_floor_type_j,other_floor_type_q,other_floor_type_s,other_floor_type_x,position_j,position_o,position_s,position_t,plan_configuration_a,plan_configuration_c,plan_configuration_d,plan_configuration_f,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
0,1020144,10,804,1371,2,5,7,6,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
1,199685,25,211,5071,2,25,9,4,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
2,1038433,22,816,12104,2,10,5,4,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
3,461920,6,449,3741,1,10,8,3,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,957027,20,254,5266,2,10,10,6,0,1,0,0,0,0,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
df_damage=pd.read_csv("train_labels.csv")
df_damage.head()

,building_id,damage_grade
0,1020144,2
1,199685,2
2,1038433,2
3,461920,2
4,957027,3


In [ ]:
df_test=pd.read_csv("test_values2.csv")
df_test = pd.get_dummies(data=df_test, columns=['land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type', 'other_floor_type', 'position', 'plan_configuration', 'legal_ownership_status'])
df_test.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,land_surface_condition_n,land_surface_condition_o,land_surface_condition_t,foundation_type_h,foundation_type_i,foundation_type_r,foundation_type_u,foundation_type_w,roof_type_n,roof_type_q,roof_type_x,ground_floor_type_f,ground_floor_type_m,ground_floor_type_v,ground_floor_type_x,ground_floor_type_z,other_floor_type_j,other_floor_type_q,other_floor_type_s,other_floor_type_x,position_j,position_o,position_s,position_t,plan_configuration_a,plan_configuration_c,plan_configuration_d,plan_configuration_f,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
0,300051,17,596,11307,3,20,7,6,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
1,99355,6,141,11987,2,25,13,5,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
2,890251,22,19,10044,2,5,4,5,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
3,745817,26,39,633,1,0,19,3,0,0,0,0,0,1,0,0,0,0,0,2,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0
4,421793,17,289,7970,3,15,8,7,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
df= pd.merge(df_train,df_damage, on='building_id',how='left')
df

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,land_surface_condition_n,land_surface_condition_o,land_surface_condition_t,foundation_type_h,foundation_type_i,foundation_type_r,foundation_type_u,foundation_type_w,roof_type_n,roof_type_q,roof_type_x,ground_floor_type_f,ground_floor_type_m,ground_floor_type_v,ground_floor_type_x,ground_floor_type_z,other_floor_type_j,other_floor_type_q,other_floor_type_s,other_floor_type_x,position_j,position_o,position_s,position_t,plan_configuration_a,plan_configuration_c,plan_configuration_d,plan_configuration_f,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w,damage_grade
0,1020144,10,804,1371,2,5,7,6,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,2
1,199685,25,211,5071,2,25,9,4,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,2
2,1038433,22,816,12104,2,10,5,4,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,2
3,461920,6,449,3741,1,10,8,3,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,2
4,957027,20,254,5266,2,10,10,6,0,1,0,0,0,0,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208475,788768,22,165,10190,2,30,5,4,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1
208476,490379,27,269,5197,3,80,5,6,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,2
208477,294228,26,1259,10829,1,10,4,6,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,3
208478,464117,7,258,1869,1,15,6,3,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,3


In [ ]:
#df = df.drop(['count_floors_pre_eq', 'has_secondary_use_use_police', 'has_secondary_use_industry', 'has_secondary_use_health_post', 'has_secondary_use_gov_office', 'plan_configuration_s'], axis=1)
#df_test = df_test.drop(['count_floors_pre_eq', 'has_secondary_use_use_police', 'has_secondary_use_industry', 'has_secondary_use_health_post', 'has_secondary_use_gov_office', 'plan_configuration_s'], axis=1)

#df = df[df.age < 201]

In [ ]:
y = df[['damage_grade']]
X = df.drop(['building_id', 'damage_grade'], axis=1)
#X = df.drop(['has_secondary_use', 'has_secondary_use_use_police', 'has_secondary_use_agriculture', 'has_secondary_use_gov_office', 'has_secondary_use_health_post',
             #'has_secondary_use_industry', 'has_secondary_use_school', 'has_secondary_use_institution', 'count_floors_pre_eq', 'legal_ownership_status_r',
             #'legal_ownership_status_a', 'legal_ownership_status_r', 'legal_ownership_status_w', 'land_surface_condition_n', 'plan_configuration_q', 'plan_configuration_n',
             #'plan_configuration_o', 'plan_configuration_f', 'plan_configuration_s', 'plan_configuration_u', 'plan_configuration_m', 'plan_configuration_d', 'plan_configuration_c', 'plan_configuration_a', 'position_o',
             #'has_superstructure_other', 'other_floor_type_x', 'land_surface_condition_o', 'ground_floor_type_z', 'has_secondary_use_hotel', 'land_surface_condition_t'], axis=1)
print(len(X.columns))

68


In [ ]:
h = .02  # step size in the mesh

names = ["Nearest Neighbors",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

X_train, X_test, y_train, y_test = train_test_split(X, y)

for name, clf in zip(names,classifiers):
  clf.fit(X_train, y_train.values.ravel())
  y_pred=clf.predict(X_test)
  f1_score = metrics.f1_score(y_test, y_pred, average='micro')
  print(name, f1_score)

Nearest Neighbors 0.6347201892914799
Decision Tree 0.4864362094779433
Random Forest 0.2413870888761135
Neural Net 0.48940675671272404
AdaBoost 0.5446514017076355
Naive Bayes 0.41857774732303404


/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning:

Variables are collinear



QDA 0.3354736020471529


In [ ]:
scaler = RobustScaler()
df_copy = df.copy()
y_copy = df[['damage_grade']]
X_copy = df.drop(['building_id', 'damage_grade'], axis=1)
#Rescaling features age, trestbps, chol, thalach, oldpeak.
features = [['age', 'count_floors_pre_eq', 'area_percentage', 'height_percentage', 'count_families']]
for feature in features:
    X_copy[feature] = scaler.fit_transform(X_copy[feature])
#Create KNN Object
knn = KNeighborsClassifier(12, weights='distance',p=1, n_jobs=-1)
#Split data
x_train, x_test, y_train, y_test = train_test_split(X_copy, y_copy.values.ravel(), test_size=0.2)
#Training the model
knn.fit(x_train, y_train)
#Predict testing set
y_pred = knn.predict(x_test)
#Check performance using f1-score
print('F1-score micro for Robust is:',metrics.f1_score(y_test, y_pred, average='micro'))

F1-score micro for Robust is: 0.7231389102072141


In [ ]:
#List Hyperparameters to tune
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]
#convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
#Making model
clf = GridSearchCV(knn, hyperparameters, cv=10)
best_model = clf.fit(x_train,y_train)
#Best Hyperparameters Value
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])
#Predict testing set
y_pred = best_model.predict(x_test)
#Check performance using accuracy
print('Accuracy:',metrics.accuracy_score(y_test, y_pred))
#Check performance using f1-score
print('F1-score micro',metrics.f1_score(y_test, y_pred, average='micro'))

KeyboardInterrupt: ignored

In [ ]:
!pip install --quiet optuna

     |████████████████████████████████| 266kB 12.2MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 7.4MB/s 
     |████████████████████████████████| 163kB 17.6MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 133kB 25.3MB/s 
     |████████████████████████████████| 112kB 32.1MB/s 
     |████████████████████████████████| 81kB 7.6MB/s 


In [ ]:
import optuna

# 1. Define an objective function to be maximized.
def objective(trial):

    # 2. Suggest values for the hyperparameters using a trial object.
    classifier_name = trial.suggest_categorical('classifier', ['KNeighborsClassifier'])
    if classifier_name == 'KNeighborsClassifier':
         p_int = trial.suggest_int('p_int',1, 15)
         classifier_obj = DecisionTreeClassifier(criterion="entropy")

    score = cross_val_score(classifier_obj, X, y, n_jobs=-1, cv=3)
    accuracy = score.mean()
    return accuracy

# 3. Create a study object and optimize the objective function.
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)
print(study.best_trial)

[I 2020-11-13 01:43:45,706] A new study created in memory with name: no-name-5890b0b8-2c63-44d7-8ed6-8c38c3d1e3ff
[I 2020-11-13 01:44:08,719] Trial 0 finished with value: 0.7135272481294285 and parameters: {'classifier': 'KNeighborsClassifier', 'leaves': 248}. Best is trial 0 with value: 0.7135272481294285.
[I 2020-11-13 01:44:39,049] Trial 1 finished with value: 0.713348827882375 and parameters: {'classifier': 'KNeighborsClassifier', 'leaves': 291}. Best is trial 0 with value: 0.7135272481294285.
[I 2020-11-13 01:44:56,031] Trial 2 finished with value: 0.7135224272728564 and parameters: {'classifier': 'KNeighborsClassifier', 'leaves': 87}. Best is trial 0 with value: 0.7135272481294285.
[I 2020-11-13 01:45:17,429] Trial 3 finished with value: 0.7135272481294285 and parameters: {'classifier': 'KNeighborsClassifier', 'leaves': 199}. Best is trial 0 with value: 0.7135272481294285.
[I 2020-11-13 01:45:38,918] Trial 4 finished with value: 0.7135272481294285 and parameters: {'classifier': '

KeyboardInterrupt: ignored

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
param_dist = {"max_depth": [3, None],
              "max_features": randint(7, 50),
              "min_samples_leaf": randint(7, 50),
              "min_samples_split": randint(2, 50),
              "criterion": ["gini", "entropy"]}

# Instantiate a Decision Tree classifier: tree
tree = DecisionTreeClassifier()

# Instantiate the RandomizedSearchCV object: tree_cv
tree_cv = RandomizedSearchCV(tree, param_dist, cv=5)

# Fit it to the data
tree_cv.fit(X_train,y_train)

# Print the tuned parameters and score
print("Tuned Decision Tree Parameters: {}".format(tree_cv.best_params_))
print("Best score is {}".format(tree_cv.best_score_))

Tuned Decision Tree Parameters: {'criterion': 'gini', 'max_depth': None, 'max_features': 48, 'min_samples_leaf': 39, 'min_samples_split': 41}
Best score is 0.7007418777180864


In [ ]:
average_f1 = []
for x in range(30):
  X_train, X_test, y_train, y_test = train_test_split(X, y)
  #clf = KNeighborsClassifier(12, weights='distance',p=1, n_jobs=-1)
  #clf = RandomForestClassifier()
  clf = DecisionTreeClassifier(criterion='gini', max_features=48, min_samples_leaf=39, min_samples_split=41)
  clf.fit(X_train, y_train.values.ravel())
  y_pred=clf.predict(X_test)
  f1_curr = metrics.f1_score(y_test, y_pred, average='micro')
  print("F1-score:",f1_curr)
  average_f1.append(f1_curr)
average_f1 = sum(average_f1) / len(average_f1)
print("Average F1-score:",average_f1)

F1-score: 0.7007482732156561
F1-score: 0.7105141980046048
F1-score: 0.702897160399079
F1-score: 0.7026669224865695
F1-score: 0.702743668457406
F1-score: 0.6997313891020721
F1-score: 0.7084420567920184
F1-score: 0.7008442056792018
F1-score: 0.7008250191864928
F1-score: 0.7016500383729855
F1-score: 0.7061588641596316
F1-score: 0.7051419800460476
F1-score: 0.7082501918649272
F1-score: 0.7114543361473522
F1-score: 0.7022448196469685
F1-score: 0.7073484267075979
F1-score: 0.7065809669992326
F1-score: 0.705487336914812
F1-score: 0.7012087490406753
F1-score: 0.705199539524175
F1-score: 0.6980621642363776
F1-score: 0.7015157329240215
F1-score: 0.7037989255564083
F1-score: 0.7020721412125863
F1-score: 0.7006715272448196
F1-score: 0.7077321565617805
F1-score: 0.7032233307751343
F1-score: 0.701016884113584
F1-score: 0.7045280122793554
F1-score: 0.703184957789716
Average F1-score: 0.7038647991813762


In [ ]:
df_test2 = df_test.drop(['building_id'], axis=1)
#df_test2 = df_test[selected_features_BE]
clf = KNeighborsClassifier(12, weights='distance',p=1, n_jobs=-1) #Highest score
# clf = KNeighborsClassifier(n_neighbors=7, leaf_size=1, weights='distance',p=1, n_jobs=-1) #Second highest score
clf.fit(X, y.values.ravel())
y_pred = clf.predict(df_test2)

In [ ]:
predictions_pd = pd.concat([df_test['building_id'], pd.DataFrame(y_pred)], axis=1)
predictions_pd = predictions_pd.rename(columns={0: "damage_grade"})
predictions_pd.to_csv('predictions.csv', index=False, float_format='%.f')
files.download("predictions.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>